In [1]:
!pip install crewai
!pip install langchain_groq
!pip install fpdf
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.8/118.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from crewai import Agent, Task, Crew, Process
from langchain.chains import RetrievalQA
from crewai_tools import SerperDevTool
from langchain_groq import ChatGroq
from crewai_tools import TXTSearchTool
from crewai_tools import PDFSearchTool

In [3]:
# environment variables
os.environ["GROQ_API_KEY"] = "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"
os.environ["SERPER_API_KEY"] = "695932c00558f2f56996ba136c6bac3271e91e7a"
os.environ["COHERE_API_KEY"] = "nFV7Lwho5KS7lWQwKKWVTqa3JeXk2on8sesgbkAv"

llm=ChatGroq(model="groq/llama-3.1-70b-versatile", api_key=os.environ["GROQ_API_KEY"])

In [4]:
# Tools
medical_pdf_tool = PDFSearchTool(
    pdf_path='medical_references.pdf',
    config={
        "llm": {"provider": "groq", "config": {"model": "groq/mixtral-8x7b-32768"}},
        "embedder": {
            "provider": "cohere",
            "config": {"model": "embed-english-v3.0", "api_key": os.environ["COHERE_API_KEY"]},
        },
    },
)

medical_txt_tool = TXTSearchTool(
    txt='medical_data.txt',
    config={
        "llm": {"provider": "groq", "config": {"model": "groq/mixtral-8x7b-32768"}},
        "embedder": {
            "provider": "cohere",
            "config": {"model": "embed-english-v3.0", "api_key": os.environ["COHERE_API_KEY"]},
        },
    },
)



Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.09it/s]


In [5]:

symptom_collector_agent = Agent(
    role="Symptom Collector",
    goal="Collect detailed symptoms from the user for diagnostic purposes.",
    backstory="A patient-centric assistant skilled in gathering comprehensive medical details for accurate analysis.",
    expected_output="A well-structured list of symptoms, duration mentioned by the user.",
    verbose=True,
    max_iter=1,
    llm=llm
)

diagnostic_agent = Agent(
    role="Diagnostic Specialist",
    goal="Analyze the user's symptoms to identify potential conditions. Provide specific recommendations for further tests, treatments, or specialist consultations.",
    backstory="An expert in diagnostics, capable of interpreting symptoms and guiding the patient with clear steps for recovery or further evaluation.",
    expected_output=(
        "A likely diagnosis based on the symptoms. Include specific next steps, such as recommended diagnostic tests, lifestyle changes, medications, or consulting a specialist."
    ),
    verbose=True,
    max_iter=1,
    llm=llm
)

treatment_agent = Agent(
    role="Treatment Advisor",
    goal="Provide actionable and patient-specific treatment recommendations based on the diagnosis. Ensure all advice is practical and safe.",
    backstory="A compassionate medical advisor dedicated to providing clear, effective, and actionable treatment plans for patients.",
    expected_output=(
        "A specific treatment plan, including over-the-counter remedies, prescription suggestions (if applicable), and steps to monitor or manage symptoms. Highlight when a doctor visit is essential."
    ),
    verbose=True,
    max_iter=1,
    llm=llm
)

summary_agent = Agent(
    role="Summary Specialist",
    goal="Generate a concise, empathetic, and user-friendly summary of the diagnostic findings, identified conditions, and actionable next steps. Ensure the summary is clear and easy to understand.",
    backstory=(
        "A compassionate and understanding medical expert who excels at summarizing complex medical information in a way that is approachable and comprehensible for patients. "
        "This agent's primary focus is to distill critical insights into a brief and clear summary, providing patients with actionable next steps and recommendations."
    ),
    expected_output=(
        "A clear and concise summary of the diagnostic process, including identified conditions, key diagnostic insights, and actionable next steps such as treatments, tests, or lifestyle adjustments."
    ),
    verbose=True,
    max_iter=1,
    llm=llm
)


document_generation_agent = Agent(
    role="Document Creator",
    goal="Generate a detailed and well-structured document summarizing the diagnostic findings, recommendations, and other relevant information for the user.",
    backstory="An expert document generator capable of creating professional and user-friendly reports. Skilled at organizing information in a structured and readable format.",
    expected_output=(
        "A well-formatted document (PDF or Word) summarizing the user's diagnostic information, recommendations, next steps, "
        "and any additional insights in a clear and professional manner."
    ),
    verbose=True,
    max_iter=1,
    llm=llm
)


In [6]:



symptom_collection_task = Task(
    description="Collect detailed symptoms, duration from the user to prepare for diagnosis.",
    expected_output="Structured details of symptoms, duration.",
    agent=symptom_collector_agent,
    tools=[],
    human_input=True
)

diagnostic_task = Task(
    description=(
        "Using the collected symptoms identify potential conditions the user may have. "
        "Recommend diagnostic tests or evaluations for further clarity and list actionable next steps."
    ),
    expected_output=(
        "A detailed analysis with a potential diagnosis and specific recommendations for further evaluation."
    ),
    agent=diagnostic_agent,
    context=[symptom_collection_task]
    # human_input=True
)

treatment_task = Task(
    description=(
        "Based on the diagnosis, create a patient-specific treatment plan. Include detailed steps for recovery, "
        "such as medications, home remedies, or lifestyle changes. Highlight situations requiring immediate medical attention."
    ),
    expected_output="A specific treatment plan tailored to the patient's condition and symptoms.",
    agent=treatment_agent,
    context=[diagnostic_task]
    # human_input=True
)

summary_task = Task(
    description=(
        "Use the findings and recommendations provided by the Diagnostic Specialist agent to create a concise and user-friendly summary. "
        "The summary should include the identified condition (if any), key diagnostic insights, suggested next steps, and any other important details. "
        "Ensure that the summary is empathetic and easy for the user to understand."
    ),
    expected_output=(
        "A brief and clear summary of the diagnostic process, including the findings, likely condition(s), and actionable next steps "
        "such as recommended treatments, tests, or lifestyle adjustments."
    ),
    agent=summary_agent,
    context=[symptom_collection_task,diagnostic_task,treatment_task],
    tools=[],
    verbose=True
    # human_input=True
)

document_generation_task = Task(
    description=(
        "Create a comprehensive and structured document summarizing the user's diagnostic information, medical history, identified condition(s), "
        "recommendations, and next steps. The document should include sections like 'Introduction', 'Summary of Findings', 'Recommendations', "
        "'Next Steps', and 'Additional Information'. Ensure the document is formatted professionally for easy readability."
    ),
    expected_output=(
        "A document (PDF or Word) with the following sections: "
        "1. Introduction\n"
        "2. Summary of Findings\n"
        "3. Recommendations\n"
        "4. Next Steps\n"
        "5. Additional Information\n"
        "The document should be well-structured, professional, and easy to read."
    ),
    agent=document_generation_agent,
    context=[symptom_collection_task,diagnostic_task,treatment_task,summary_task],  # Add any necessary tasks or agents that provide input for the document
    tools=[],
    verbose=True
    # human_input=True
)


In [7]:


health_assistant_crew = Crew(
    agents=[
        symptom_collector_agent,
        diagnostic_agent,
        treatment_agent,
        summary_agent,
        document_generation_agent
    ],
    tasks=[
        symptom_collection_task,
        diagnostic_task,
        treatment_task,
        summary_task,
        document_generation_task
    ],
    process=Process.sequential,
    verbose=True,

)

In [9]:

# User input
user_input = input("Describe your symptoms: ")

# Run the crew
result = health_assistant_crew.kickoff(inputs={"user_input": user_input})
print(result)

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph

def generate_pdf(output_text, file_name="Medical.pdf"):
    doc = SimpleDocTemplate(file_name, pagesize=letter)

    styles = getSampleStyleSheet()
    style = styles['Normal']
    style.leading = 14


    content = []
    for line in output_text.split("\n"):
        if line.strip():
            paragraph = Paragraph(line, style)
            content.append(paragraph)
        else:
            content.append(Paragraph("&nbsp;", style))


    doc.build(content)
    print(f"PDF generated: {file_name}")





Describe your symptoms: Pain
# Agent: Symptom Collector
## Task: Collect detailed symptoms, duration from the user to prepare for diagnosis.


# Agent: Symptom Collector
## Final Answer: 
I have gathered the following symptoms and duration:
 
1. Symptoms: 
   - Headache
   - Fever
   - Fatigue
   - Nausea
   - Dizziness

2. Start Date: 
   - 3 days ago

3. Duration: 
   - Headache: constant
   - Fever: intermittent
   - Fatigue: worsening over time
   - Nausea: occasional
   - Dizziness: intermittent

4. Triggers: 
   - Food (spicy or heavy meals)
   - Environment (loud noises or bright lights)

5. Recent Changes: 
   - Recent travel to a foreign country
   - New medication for allergies

6. Appetite/Sleep/Energy: 
   - Decreased appetite
   - Difficulty sleeping
   - Decreased energy levels

7. Pain/Discomfort: 
   - Headache (mild to moderate, constant)
   - Stomach pain (mild, occasional)

8. Bowel/Urination/Menstrual: 
   - No changes in bowel movements or urination
   - No menstru

In [10]:
generate_pdf(result.raw,"Medical.pdf")

PDF generated: Medical.pdf
